In [1]:
import pandas as pd
import regex as re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/ranggi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data MUA for Memory Based System Recommendation

In [2]:
class ContentBasedFiltering:
    def case_folding(self, text):
        if type(text) == str:
            text = text.lower()
        return text

    def punctuation_removal(self, text):
        if type(text) == str:
            text = re.sub(r'[^\w\s]', '', text)
        return text

    def tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens
        
    def stopwords_removal(self, text):
        stopword_factory = StopWordRemoverFactory()
        stopword_remover = stopword_factory.create_stop_word_remover()
        no_stopword = stopword_remover.remove(text)
        return no_stopword

    def preprocessing(self, text):
        text = self.case_folding(text)
        text = self.punctuation_removal(text)
        text = self.stopwords_removal(text)
        return text

In [3]:
data = pd.read_excel('./web_mua/static/Data MUA 2.xlsx')
data.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating
0,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,natural (putih),light,cool,Murah,-8.233613,113.66733,4
1,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,4
2,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,rachel 1 (kuning langsat),medium,neutral,Murah,-8.233613,113.66733,5
3,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,5
4,3,septia maharani,Rias Pengantin Azizah,-8.234863,113.680670,la tulipe cover foundation,natural (putih),light,cool,Sedang,-8.233613,113.66733,3


In [4]:
def cleaning_data(data):
    cleaned_data = []
    CBF = ContentBasedFiltering()
    for i in range(len(data)):
        cleaned_data.append(CBF.preprocessing(str(data[i])))
    return cleaned_data

In [5]:
#praproses data untuk nama
data['nama'] = cleaning_data(data['nama'])

In [6]:
data_for_model = data.copy()
data_for_user = data.copy()

#praproses data untuk deskripsi
data_for_model['desc'] = data['produk_makeup'] + ' ' + data['shade'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_model['desc'] = cleaning_data(data_for_model['desc'])
data_for_user['desc'] = data['produk_makeup'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_user['desc'] = cleaning_data(data_for_user['desc'])

In [7]:
# data_for_model = data_for_model.groupby('no').agg({'nama':'first', 'nama_MUA':'first', 'kategori_harga': 'first', 'latitude':'first', 'longtitude':'first', 'rating':'first','desc':' '.join}).reset_index()
data_for_model = data_for_model.sort_values(by=['nama'], ignore_index=True)

data_for_user = data_for_user.sort_values(by=['nama'], ignore_index=True)
data_for_user = data_for_user.drop(columns=['shade'])

In [8]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [9]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


In [10]:
data_for_model.shape

(179, 14)

In [11]:
from sklearn.preprocessing import LabelEncoder

harga_labels = LabelEncoder()
harga_labels = harga_labels.fit(data['kategori_harga'])
harga_labels.classes_ = np.array(['Murah', 'Sedang', 'Mahal'])
data_for_model['kategori_harga'] = harga_labels.transform(data_for_model['kategori_harga'])
data_for_user['kategori_harga'] = harga_labels.transform(data_for_user['kategori_harga'])

In [12]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [13]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


## User Input

In [14]:
# use this parameter to get some MUA recommendations
def data_preparation(data_for_model, data_for_user, index_user, user):
    du = data_for_user.copy()
    query_data = du.iloc[[index_user]].reset_index()
    query_data_todict = query_data.to_dict('index')[0]
    part_of_model = data_for_model.copy()
    part_of_model = part_of_model.loc[part_of_model.nama_MUA != query_data.nama_MUA[0]].reset_index()
    part_of_query = part_of_model.loc[part_of_model['nama'] == user].reset_index()
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return query_data_todict, part_of_query, part_of_model

# example of input data
find_user = data_for_user.index[data_for_user['nama'] == 'adera zia'].to_list()
txt_user = 'adera zia'
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user)
# part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user, first=False)

## Calculating Cosine Similarity for Content Based Model

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from sklearn.decomposition import TruncatedSVD

def tf_idf_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(md["desc"])
    model = X[:len(part_of_model)]
    query = X[len(part_of_model):]
    return model, query

def doc2vec_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    words_split = []
    for i in range(len(md)):
        words_split.append([word for word in md["desc"][i].split()])
    documents = []
    for i, list_of_words in enumerate(words_split):
        documents.append(gensim.models.doc2vec.TaggedDocument(words=list_of_words, tags=['doc'+str(i)]))
    model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=30)
    # vectorizer = TfidfVectorizer()
    # X = vectorizer.fit_transform(md["desc"])
    # nmf_model = NMF(10)
    # doc_topic = nmf_model.fit_transform(X)
    vec_model = []
    for i in range((len(part_of_model))):
        vec_model.append(model.infer_vector(md["desc"][i].split()))
    vec_query = model.infer_vector(md["desc"][len(md["desc"])-1].split())
    # print(vec_query)
    return vec_model, vec_query

In [16]:
import haversine as hs
import math
def calculate_distance(query_data_todict, part_of_model):
        loc1 = (query_data_todict['latitude'], query_data_todict['longtitude'])
        distance_loc = []
        for i in range(len(part_of_model.index)):
            loc2 = (part_of_model['latitude'][i], part_of_model['longtitude'][i])
            tmp = hs.haversine(loc1,loc2)
            value = math.ceil(tmp * 100) / 100
            distance_loc.append(value)
        return distance_loc

In [50]:
from gensim import similarities
from scipy import spatial 
import torch

def doc2vec_model(sentences1, sentences2, model):
    res = []
    for sentence1, sentence2 in zip(sentences1, sentences2):
        res.append(model.similarity_unseen_docs(sentence1, sentence2))
    return torch.tensor(res)

def get_numtext_content(query_data_todict, part_of_model, tfidf=False):
    if tfidf == False:
        vec_model, vec_query = doc2vec_vector(part_of_model, query_data_todict)
        cos_sim = []
        for i in range(len(vec_model)):
            cos_sim.append(spatial.distance.cosine(vec_model[i], vec_query))
        distance = calculate_distance(query_data_todict, part_of_model)
        return cos_sim, distance
    else:
        tfidf_model, tfidf_query = tf_idf_vector(part_of_model, query_data_todict)
        cos_sim = cosine_similarity(tfidf_query, tfidf_model)
        data_sim = cos_sim.reshape(-1,1)
        distance = calculate_distance(query_data_todict, part_of_model)
        return data_sim, distance

In [51]:
# x, y = get_numtext_content(query_data_todict, part_of_model)

In [52]:
from sklearn.preprocessing import StandardScaler

def get_values(part_of_model):
    tmp_data = part_of_model[['kategori_harga','latitude', 'longitude']].to_numpy()
    sim_num = cosine_similarity(tmp_data)
    sim_num = np.array([np.mean(p) for p in sim_num])
    return sim_num

In [53]:
import statistics 
def value_of_mua(query_data_todict, part_of_model, tfidf=False):
    sim, distance = get_numtext_content(query_data_todict, part_of_model, tfidf)
    part_of_model["sim"] = sim
    part_of_model["distance"] = distance
    sim_num = get_values(part_of_model)
    part_of_model["sim_num"] = sim_num
    part_of_model = part_of_model.sort_values(by=['nama_MUA'])
    nama_mua = part_of_model["nama_MUA"].unique()
    value_mua = np.empty(len(nama_mua), dtype=float)
    for i in range(len(nama_mua)):
        tmp_val = []
        for j in range(len(part_of_model.index)):
            if nama_mua[i] == part_of_model.nama_MUA[j]:
                tmp_val.append(statistics.fmean([part_of_model["sim_num"][j], part_of_model["sim"][j]]))
        value_mua[i] = np.mean(tmp_val)
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return nama_mua, value_mua, part_of_model

## FIND USER

In [54]:
unique_user = data_for_model['nama'].unique()
unique_user
query_user = ['adera zia', 'anggia pradita a', 'anissa serina', 'calista devi aqilah', 'callista febriana erwanto', 'dwi rahma', 'maulina vira', 
              'vika aprilia sinta putri', 'yuricha mahargiyani wijaya', 'zainata hanum masruroh', 'zyanti laila']
selected_user = []
for i in query_user:
    find_user = data_for_model.index[data_for_model['nama'] == i].to_list()
    selected_user.append(find_user[0])
selected_user

[0, 10, 20, 33, 45, 73, 101, 142, 151, 161, 169]

## HYBRID FILTERING

In [55]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing as pre

In [56]:
#chosen user for testing
def hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user, tfidf=False):
    actual_hyb = []
    table_hyb = []
    rating_hyb = []
    for index_user in range(len(selected_user)):
        query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model, tfidf)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()

        for i in range(len(val_model)):
            for j in range(len(val_model[i])):
                if val_model[i][j] == 0:
                    for z in range(len(nama_mua)):
                        if col_model[j] == nama_mua[z]:
                            val_model[i][j] = value_mua[j]
                            break

        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        
        for i in range(len(feature)):
            if feature[i] == 0.0:
                for j in range(len(nama_mua)):
                    if col_model[i] == nama_mua[j]:
                        feature[i] = value_mua[j]
                        break

        cos_sim_user = cosine_similarity([feature], val_model)

        weight_user = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_user)):
                if part_of_model["nama"][i] == nama_user[j]:
                    weight_user[i] = cos_sim_user[0][j]
        
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA']]
        final_res['user'] = weight_user
        final_res['content'] = weight_content
        
        mean_weight = np.zeros(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            mean_weight[i] = weight_user[i]
        final_res['mean_'] = mean_weight
        higher_value = final_res.copy()
        higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
        higher_value['weight_user'] = u['mean']
        c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
        higher_value['weight_content'] = c['mean']
        higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
        list_nama = higher_value.nama[:15].tolist()
        pivot = part_of_model.copy()
        pivot['mean_'] = mean_weight
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_['weight']
        pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        if pivot_['weight'][i] != 0.0:
                            value_rating[j] = pivot_["mean"][i] * pivot_['weight'][i]
                        else:
                            value_rating[j] = weight_content[i]
                else:
                    continue
        
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break

        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        
        # for i in range(len(final_label)):
        #     if final_label[i] == 0.0:
        #         for j in range(len(pivot_['mean'])):
        #             if col_model[i] == pivot_.index.tolist()[j]:
        #                 final_label[i] = pivot_['mean'].tolist()[j]
        actual_hyb.append(final_label)
        table_hyb.append(pivot_)
        rating_hyb.append(value_rating)

    #     mae = mean_absolute_error(value_rating, final_actual[index_user])
    #     mae_hy.append(mae)
        
    # mean_mae_hy = np.mean(mae_hy)
    return table_hyb, rating_hyb, actual_hyb, col_model

In [57]:
hybrid_based_fil_doc2vec, value_rating_hyb_doc2vec, actual_hyb_doc2vec, col_model_doc2vec = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [58]:
hybrid_based_fil_tfidf, value_rating_hyb_tfidf, actual_hyb_tfidf, col_model_tfidf = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user, tfidf=True)

## COLLABORATIVE FILTERING

In [59]:
# def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
#     actual_colab = []
#     table_colab = []
#     rating_colab = []
#     for index_user in range(len(selected_user)):
#         _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
#         nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
#         pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         col_model = pivot.columns.tolist()
#         val_model = np.array(pivot.values.tolist(), dtype=float)
#         nama_user = np.array(pivot.index.tolist())
#         pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         val_query = pivot_query.values.tolist()
#         col_query = pivot_query.columns.tolist()
#         feature = np.zeros(len(col_model), dtype=float)
#         for i in range(len(col_model)):
#             for j in range(len(col_query)):
#                 if col_model[i] == col_query[j]:
#                     feature[i] = val_query[0][j]
#                     break
        
#         cos_sim_user = cosine_similarity([feature], val_model)
        
#         final_res = part_of_model.copy()
#         final_res = final_res[['nama', 'nama_MUA', 'rating']]
#         tmp_cos = np.zeros(len(part_of_model), dtype=float)
#         for i in range(len(final_res)):
#             for j in range(len(nama_user)):
#                 if final_res['nama'][i] == nama_user[j]:
#                     tmp_cos[i] = cos_sim_user[0][j]
#                     break

#         final_res['mean_'] = tmp_cos
#         higher_value = final_res.copy()
#         higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
#         list_nama = higher_value.nama[:15].tolist()
#         pivot = final_res.copy()
#         pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
#         pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
#         pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
#         pivot_['predicted'] = pivot_["mean"] * pivot_['weight']
#         pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
#         value_rating = np.zeros(len(col_model), dtype=float)
#         for i in range(len(pivot_.index)):
#             for j in range(len(col_model)):
#                 if value_rating[j] == 0.0:
#                     if col_model[j] == pivot_.index[i]:
#                         value_rating[j] = pivot_["mean"][i] * pivot_["weight"][i]
#                 else:
#                     continue
#         tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
#         mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
#         pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         val_label = pivot_label.values.tolist()
#         col_label = pivot_label.columns.tolist()
#         final_label = np.zeros(len(col_model), dtype=float)
#         for i in range(len(col_model)):
#             for j in range(len(col_label)):
#                 if col_model[i] == col_label[j]:
#                     final_label[i] = val_label[0][j]
#                     break
#         # mae = mean_absolute_error(value_rating, final_actual[index_user])
#         # mae_col.append(mae)

#         # for i in range(len(final_label)):
#         #     if final_label[i] == 0.0:
#         #         for j in range(len(pivot_['mean'])):
#         #             if col_model[i] == pivot_.index.tolist()[j]:
#         #                 final_label[i] = pivot_['mean'].tolist()[j]
#         actual_colab.append(final_label)
#         table_colab.append(pivot_)
#         rating_colab.append(value_rating)
        
#     # mean_mae_col = np.mean(mae_col)
#     return table_colab, rating_colab, actual_colab

In [60]:
def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
    actual_colab = []
    table_colab = []
    rating_colab = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        knn = NearestNeighbors(metric='cosine', algorithm='brute')
        knn.fit(val_model)
        distances, indices = knn.kneighbors([feature], n_neighbors=10)
        list_nama = []
        for ln in indices[0]:
             list_nama.append(nama_user[ln])
        pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
        tmp_cos = np.empty(len(pivot), dtype=float)
        for i in range(len(pivot.nama)):
            for j in range(len(list_nama)):
                if pivot.nama[i] == list_nama[j]:
                    tmp_cos[i] = distances[0][j]
        pivot['cos'] = tmp_cos
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
        pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_col.append(mae)

        # for i in range(len(final_label)):
        #     if final_label[i] == 0.0:
        #         for j in range(len(pivot_['mean'])):
        #             if col_model[i] == pivot_.index.tolist()[j]:
        #                 final_label[i] = pivot_['mean'].tolist()[j]
        actual_colab.append(final_label)
        table_colab.append(pivot_)
        rating_colab.append(value_rating)
        
    # mean_mae_col = np.mean(mae_col)
    return table_colab, rating_colab, actual_colab

In [61]:
colab_based_fil, value_rating_colab, actual_colab = colab_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [62]:
value_rating_colab[0]

array([0.        , 0.        , 0.        , 1.4220351 , 0.        ,
       2.83294725, 0.        , 1.9819781 , 4.06295743, 0.        ,
       5.        , 0.        , 3.16438173, 0.        , 0.        ,
       0.        , 2.07726426, 0.        , 0.        , 1.731989  ,
       2.97828796, 3.17216767, 0.        , 3.71886526, 0.        ,
       0.        , 4.06295743, 0.        , 0.        , 0.        ,
       2.43777446, 5.        , 0.        , 4.06295743, 0.        ,
       0.        , 0.        , 0.        , 2.13439715, 0.        ,
       0.        , 0.        , 2.76968568, 0.        , 3.68875082,
       3.55732858, 3.68680064, 2.5376995 , 0.        , 0.        ,
       0.        ])

In [63]:
# colab_based_fil[colab_based_fil.score > 0.0]

In [64]:
colab_based_fil[0]

,count,mean,cos,predicted
nama_MUA,,,,
Rias Pengantin Sapphire,2,5.000000,1.000000,5.000000
Gasebo Make Up Dekorasi,2,5.000000,1.000000,5.000000
Eka Make Over,3,5.000000,0.812591,4.062957
Rias Manten Mariyam,3,5.000000,0.812591,4.062957
Salon Fatah,2,5.000000,0.812591,4.062957
Nonarika Make Up & Hairdo,3,4.000000,0.929716,3.718865
Ulfa Beauty MUA,3,4.666667,0.790447,3.688751
Winda Wijaya MUA & Beauty Studio,3,5.000000,0.737360,3.686801
Wedding Gallery Make Over,2,5.000000,0.711466,3.557329


## CONTENT BASED FILTERING

In [65]:
def content_evaluation(data_for_model, data_for_user, selected_user, query_use, tfidf=False):
    actual_content = []
    table_content = []
    rating_content = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model, tfidf)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        print(value_mua)
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]

        pm = part_of_model.copy()
        pm["cos"] = weight_content
        pivot_ = pm.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pm.groupby("nama_MUA").cos.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
        pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
        pivot_ = pivot_[:15]
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_con.append(mae)

        # for i in range(len(final_label)):
        #     if final_label[i] == 0.0:
        #         for j in range(len(pivot_['mean'])):
        #             if col_model[i] == pivot_.index.tolist()[j]:
        #                 final_label[i] = pivot_['mean'].tolist()[j]
        actual_content.append(final_label)
        table_content.append(pivot_)
        rating_content.append(value_rating)
        
    # mean_mae_con = np.mean(mae_con)
    return table_content, rating_content, actual_content

In [66]:
content_based_fil, value_rating_content, actual_content = content_evaluation(data_for_model, data_for_user, selected_user, query_user, tfidf=True)

[0.66875634 0.52351057 0.69093082 0.59356596 0.5606445  0.65316664
 0.53504911 0.66074346 0.65409829 0.5187358  0.7525199  0.52936289
 0.53593664 0.53140073 0.58909355 0.53462583 0.77373778 0.55076355
 0.68865649 0.62704293 0.62704309 0.51848812 0.63625318 0.54000505
 0.68865629 0.61118098 0.67381211 0.76696408 0.75251996 0.66472313
 0.64587269 0.68391354 0.63628455 0.65842443 0.53462582 0.53462584
 0.63595435 0.62256491 0.5673197  0.53462543 0.58666424 0.661459
 0.65256925 0.58911564 0.61117864 0.57052649 0.56393881 0.77373781
 0.53462584 0.53989287 0.61131394]
[0.66729074 0.52400425 0.69072236 0.59270571 0.65281997 0.53561318
 0.66077578 0.70822251 0.65350008 0.51914186 0.75229581 0.52998818
 0.53657398 0.531946   0.58918868 0.53523938 0.77338766 0.55159209
 0.6885564  0.62722804 0.62722818 0.51888672 0.63503137 0.54069028
 0.6885562  0.61069899 0.67285411 0.76689723 0.75229586 0.66453029
 0.64562674 0.68272252 0.63506128 0.65844251 0.53523939 0.53523941
 0.63472182 0.62367174 0.5666

In [67]:
# content_based_fil[content_based_fil.score > 0.0]

In [68]:
content_based_fil[0]

,count,mean,cos,predicted
nama_MUA,,,,
Gasebo Make Up Dekorasi,4,5.000000,0.752520,3.762599
Khaliza Make Up,3,5.000000,0.688656,3.443282
Rias Pengantin Sapphire,4,5.000000,0.683914,3.419568
Abha Beauty Make Up Artist,4,5.000000,0.668756,3.343782
Sanggar Rias Nining,5,5.000000,0.661459,3.307295
Rias Manten Mariyam,5,4.600000,0.673812,3.099536
Yasmin MUA dan Henna Jember,2,4.000000,0.773738,3.094951
Rias Ning Lida,2,4.000000,0.766964,3.067856
RR Make Up,3,5.000000,0.611181,3.055905


In [69]:
merge_actual = [[i, j, z] for i,j,z in zip(actual_colab, actual_content, actual_hyb_tfidf)]
final_actual = []
for i in range(len(merge_actual)):
    tmp = []
    for j in range(len(merge_actual[i][0])):
        sfmean = statistics.fmean([merge_actual[i][0][j], merge_actual[i][1][j], merge_actual[i][2][j]])
        tmp.append(round(sfmean))
    final_actual.append(tmp)
    

In [70]:
def mae_evaluation(predicted_data, actual_data, hybrid=False):
    maeUser = []
    rmseUser = []

    for i in range(len(predicted_data)):
        mae = mean_absolute_error(predicted_data[i], actual_data[i])
        rmse = mean_squared_error(predicted_data[i], actual_data[i], squared=False)
        maeUser.append(mae)
        rmseUser.append(rmse)
    mae = np.mean(maeUser)
    rmse = np.mean(rmseUser)
    if hybrid == True:
        return mae, rmse, maeUser
    else:
        return mae, rmse

In [71]:
mae_col, rmse_col = mae_evaluation(value_rating_colab, final_actual)
mae_con, rmse_con = mae_evaluation(value_rating_content, final_actual)
mae_hyb, rmse_hyb, mae = mae_evaluation(value_rating_hyb_doc2vec, final_actual, hybrid=True)

In [72]:
data_evaluasi = {
    'Metode' : ['Collaborative Filtering', 'Content Based Filtering', 'Hybrid Filtering'],
    'MAE' : [mae_col, mae_con, mae_hyb],
    'RMSE' : [rmse_col, rmse_con, rmse_hyb]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE,RMSE
0,Hybrid Filtering,0.729048,1.191719
1,Content Based Filtering,1.135301,1.944058
2,Collaborative Filtering,1.188183,2.065942


In [73]:
mae_hyb_tfidf, rmse_hyb_tfidf, mae_tfidf = mae_evaluation(value_rating_hyb_tfidf, final_actual, hybrid=True)
data_evaluasi_hyb = {
    'Metode' : ['TFIDF', 'Doc2vec'],
    'MAE' : [mae_hyb_tfidf, mae_hyb],
    'RMSE' : [rmse_hyb_tfidf, rmse_hyb]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi_hyb)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE,RMSE
0,Doc2vec,0.729048,1.191719
1,TFIDF,0.791273,1.320110


In [74]:
def testing_user(data_for_model, data_for_user, index_usr):
    query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_usr], query_user[index_usr])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    
    for i in range(len(val_model)):
        for j in range(len(val_model[i])):
                if val_model[i][j] == 0:
                    for z in range(len(nama_mua)):
                        if col_model[j] == nama_mua[z]:
                            val_model[i][j] = value_mua[j]
                            break
    
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    
    for i in range(len(feature)):
            if feature[i] == 0.0:
                for j in range(len(nama_mua)):
                    if col_model[i] == nama_mua[j]:
                        feature[i] = value_mua[j]
                        break

    cos_sim_user = cosine_similarity([feature], val_model)

    # nmf_model = NMF(3)
    # f = nmf_model.fit_transform([feature])
    # m = nmf_model.fit_transform(val_model)
    # cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]
        
    weight_content = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_mua)):
            if part_of_model["nama_MUA"][i] == nama_mua[j]:
                weight_content[i] = value_mua[j]
        
    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user
    final_res['content'] = weight_content

    mean_weight = np.zeros(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        mean_weight[i] = weight_user[i]
    final_res['mean_'] = mean_weight
    higher_value = final_res.copy()
    higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
    u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
    higher_value['weight_user'] = u['mean']
    c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
    higher_value['weight_content'] = c['mean']
    higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
    print(higher_value)
    list_nama = higher_value.nama[:15].tolist()
    pivot = part_of_model.copy()
    pivot['mean_'] = mean_weight
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    distance = pivot[['nama_MUA','distance']]
    produk = pivot[['nama_MUA', 'sim']]
    desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
    desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
    pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
    pivot_['score'] = pivot_["mean"] * pivot_['weight']
    pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
    pivot_['distance'] = distance.groupby("nama_MUA").distance.agg(['mean'])
    pivot_['sim'] = produk.groupby("nama_MUA").sim.agg(['mean'])
    pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)
    return pivot_

In [75]:
#highest MAE
mae[mae.index(min(mae))]

0.5349187437060322

In [76]:
#highest
idx_high = mae.index(min(mae))
data_for_model.iloc[[selected_user[idx_high]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
45,43,callista febriana erwanto,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,0,-8.157255,113.652074,4,la tulipe cover foundation suntan sawo matang ...


In [77]:
h_pivot = testing_user(data_for_model, data_for_user, idx_high)
h_pivot

                              nama      mean  weight_user  weight_content
0        callista febriana erwanto  1.000000     1.000000        0.516959
1                    eka febrianti  0.738413     0.738413        0.512913
2               rodhiyyatul hikmah  0.726991     0.726991        0.529518
3                  indi miftahul h  0.553238     0.553238        0.516354
4                    nadia safitri  0.552989     0.552989        0.510157
5       yuricha mahargiyani wijaya  0.513899     0.513899        0.519108
6          ulifah azwarani rochmah  0.499097     0.499097        0.524906
7                  septia maharani  0.498816     0.498816        0.519668
8                 hikmatul hasanah  0.498620     0.498620        0.516037
9                  diajeng agustin  0.498544     0.498544        0.514620
10              trevina wigianiska  0.498514     0.498514        0.514065
11                     dea wanda p  0.498470     0.498470        0.513254
12           arsya putri rifasanti  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim
0,5,4.600000,Rias Manten Mariyam,0.895365,4.118680,[la tulipe cover foundation - rachel 1 (kuning...,1.91,0.025845
1,5,4.600000,Eka Make Over,0.805559,3.705573,[la tulipe cover foundation - natural (putih) ...,54.12,0.025833
2,7,4.571429,Salon Fatah,0.698330,3.192367,[la tulipe cover foundation - suntan (sawo mat...,33.67,0.059060
3,3,3.000000,Rias Pengantin Criepno Lestari,1.000000,3.000000,[la tulipe cover foundation - suntan (sawo mat...,28.71,0.033386
4,3,5.000000,Khaliza Make Up,0.478110,2.390551,[la tulipe cover foundation - natural (putih) ...,23.39,0.036992
5,2,4.000000,Laura Make Up & Bridal,0.513899,2.055594,[la tulipe cover foundation - suntan (sawo mat...,5.34,0.052883
6,6,4.000000,Yulia Salon & Bridal,0.488365,1.953461,[la tulipe cover foundation - natural (putih) ...,25.48,0.034012
7,3,4.000000,Novi Make Up,0.478110,1.912441,[LT Pro smooth corrector cream foundation - na...,28.29,0.034214
8,4,3.500000,Ayunia Bridal dan Salon,0.533568,1.867489,[la tulipe cover foundation - suntan (sawo mat...,6.22,0.032743
9,2,3.000000,Dinda Wijaya Make Up,0.513899,1.541696,[la tulipe cover foundation - suntan (sawo mat...,6.70,0.096705


In [45]:
h_pivot.to_excel('highest2.xlsx')

In [78]:
#lowest MAE
mae[mae.index(max(mae))]

0.8831624408880742

In [79]:
#lowest
idx_low = mae.index(max(mae))
data_for_model.iloc[[selected_user[idx_low]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
151,23,yuricha mahargiyani wijaya,Ayunia Bridal dan Salon,-8.17887,113.69919,Wardah Lightening Liquid Foundation 25ml,neutral beige,light,warm,0,-8.17887,113.696615,5,wardah lightening liquid foundation 25ml neutr...


In [80]:
h_pivot = testing_user(data_for_model, data_for_user, idx_low)
h_pivot

                              nama      mean  weight_user  weight_content
0       yuricha mahargiyani wijaya  1.000000     1.000000        0.516444
1                     zyanti laila  0.641794     0.641794        0.519175
2                    nadia safitri  0.627477     0.627477        0.506161
3                    anissa serina  0.603931     0.603931        0.515791
4                       yuna ayana  0.568024     0.568024        0.524087
5                  septia maharani  0.567978     0.567978        0.523431
6                      dea wanda p  0.567442     0.567442        0.515796
7                  diajeng agustin  0.567403     0.567403        0.515242
8          ulifah azwarani rochmah  0.567078     0.567078        0.510640
9                 hikmatul hasanah  0.567027     0.567027        0.509916
10           arsya putri rifasanti  0.566996     0.566996        0.509476
11              trevina wigianiska  0.566989     0.566989        0.509375
12       callista febriana erwanto  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim
0,5,4.600000,Eka Make Over,0.739342,3.400973,[la tulipe cover foundation - natural (putih) ...,58.99,0.016165
1,5,4.200000,Laura Make Up & Bridal,0.777504,3.265516,[la tulipe cover foundation - natural (putih) ...,2.22,0.025062
2,2,5.000000,Salon Fatah,0.565570,2.827850,[LT Pro smooth corrector cream foundation - na...,34.88,0.019939
3,3,5.000000,Rias Manten Mariyam,0.565570,2.827850,[la tulipe cover foundation - rachel 1 (kuning...,5.68,0.021391
4,6,3.666667,Dinda Wijaya Make Up,0.735288,2.696056,[LT Pro smooth corrector cream foundation - na...,8.94,0.057522
5,2,4.000000,Yasmin MUA dan Henna Jember,0.641794,2.567177,[la tulipe cover foundation - rachel 1 (kuning...,7.94,0.016339
6,2,4.000000,Ayuk Rias Pengantin,0.641794,2.567177,[Maybelline Fit Me Matte + Poreless Liquid Fou...,17.11,0.053506
7,5,4.200000,Larasnada Make Up,0.603132,2.533153,[la tulipe cover foundation - natural (putih) ...,8.35,0.055402
8,1,4.000000,Ulfa Beauty MUA,0.603931,2.415723,[la tulipe cover foundation - suntan (sawo mat...,43.89,0.078789
9,1,4.000000,Griya Putri Make Up,0.603931,2.415723,[la tulipe cover foundation - natural (putih) ...,2.75,0.013574


In [49]:
h_pivot.to_excel('lowest2.xlsx')

In [ ]:
# rating_test = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = final_actual[i][j]
#     rating_test.append(rating_dict)

# dataTesting = pd.DataFrame.from_dict(rating_test)
# dataTesting['user'] = query_user
# dataTesting.set_index(['user'])
# dataTesting.to_excel('dataTesting2.xlsx')

In [ ]:
# predicted_rating = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = value_rating_hyb[i][j]
#     predicted_rating.append(rating_dict)

# predictedRating = pd.DataFrame.from_dict(predicted_rating)
# predictedRating['user'] = query_user
# predictedRating.set_index(['user'])
# predictedRating.to_excel('predictedRating2.xlsx')